In [1]:
import tensorflow as tf
# import numpy
import random
import math
import time

In [3]:
# tf Graph Input
num_inputs = 32
num_hidden_1 = num_inputs*2
num_hidden_2 = num_inputs*2
num_outputs = 1
stddev = 1.0 

# with tf.device("/gpu:0"):
if True: 
    inputs = tf.placeholder(shape=[None, num_inputs], name="inputs", dtype=tf.float16)
    expected = tf.placeholder(shape=[None, num_outputs], name="expected", dtype=tf.float16)

    # First hidden layer of the network 
    h1w = tf.Variable(expected_shape=[num_inputs, num_hidden_1], name="hidden_1_weight", 
                     initial_value=tf.truncated_normal([num_inputs, num_hidden_1], stddev=stddev, dtype=tf.float16), dtype=tf.float16)
    h1b = tf.Variable(expected_shape=[1, num_hidden_1], name="hidden_1_bias", 
                     initial_value=tf.truncated_normal([1, num_hidden_1], stddev=stddev, dtype=tf.float16), dtype=tf.float16)

    h1l = tf.nn.sigmoid(tf.add(tf.matmul(inputs, h1w), h1b))

    # Second hidden layer of the network 
    h2w = tf.Variable(expected_shape=[num_hidden_1, num_hidden_2], name="hidden_2_weight", 
                     initial_value=tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=stddev, dtype=tf.float16), dtype=tf.float16)
    h2b = tf.Variable(expected_shape=[1, num_hidden_2], name="hidden_2_bias", 
                     initial_value=tf.truncated_normal([1, num_hidden_2], stddev=stddev, dtype=tf.float16), dtype=tf.float16)

    h2l = tf.nn.sigmoid(tf.add(tf.matmul(h1l, h2w), h2b))

    # Weight and biases used to turn hidden layer into final output value. 
    ow = tf.Variable(expected_shape=[num_hidden_2, num_outputs], name="output_weight", 
                     initial_value=tf.truncated_normal([num_hidden_2, num_outputs], stddev=stddev, dtype=tf.float16), dtype=tf.float16)
    ob = tf.Variable(expected_shape=[1, num_outputs], name="output_bias", 
                     initial_value=tf.truncated_normal([1, num_outputs], stddev=stddev, dtype=tf.float16), dtype=tf.float16)

    output = tf.nn.sigmoid(tf.add(tf.matmul(h2l, ow), ob))

    # We compute error as the mean square difference 
    error = tf.reduce_mean(tf.pow(tf.subtract(output, expected), 2))


# Parameters
learning_rate = 0.025

# Gradient descent
# optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(error)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(error)

training_steps = 500000

min = 1
max = 2**num_inputs
parity_max = 6

display_step = 500
batch_size = 1000

def binlist(x, length):
    return [1.0 * (0!=(x&(1<<b))) for b in range(length)]

def onehot(x, length):
    return [1.0 * (x==b) for b in range(length)]

def lesshot(x, length):
    return [1.0 * (x<=b) for b in range(length)]

def zero(length): return [0] * length 

def makeinput(x):
    return binlist(x, num_inputs)

def f(x):  
    binx = binlist(x, num_inputs)
    numbits = sum([binx[i] for i in range(parity_max)])
    return [numbits%2,]

# Launch the graph
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    tf_globals = tf.global_variables_initializer()
    sess.run(tf_globals)
    t = time.clock() 
    for epoch in range(training_steps):
        data = { inputs: [], expected: []}
        for b in range(batch_size):
            x = random.randint(min,max)
            data[inputs].append(makeinput(x))
            data[expected].append(f(x))
        sess.run(optimizer, feed_dict=data)

        if epoch % display_step == 0:
            
            e = sess.run(error, feed_dict=data)
            print "Epoch:", epoch, " error=", e, "(",time.clock() - t,")"
            t = time.clock() 
    for x in range(min, parity_max):
        data = {inputs: [makeinput(x)], expected: [f(x)]}
        print (" x=", x, " output=", sess.run(output, feed_dict=data), 
        " expected=", data[expected], 
        " err=",  sess.run(error, feed_dict=data))

Epoch: 0  error= 0.47998 ( 0.064278 )
Epoch: 500  error= 0.25391 ( 13.574678 )
Epoch: 1000  error= 0.25122 ( 13.42506 )
Epoch: 1500  error= 0.24951 ( 13.401728 )
Epoch: 2000  error= 0.25024 ( 13.831343 )
Epoch: 2500  error= 0.25 ( 13.764169 )
Epoch: 3000  error= 0.25293 ( 13.628893 )
Epoch: 3500  error= 0.25146 ( 13.560731 )
Epoch: 4000  error= 0.26074 ( 13.614717 )
Epoch: 4500  error= 0.25952 ( 13.518145 )
Epoch: 5000  error= 0.25269 ( 13.593555 )
Epoch: 5500  error= 0.25195 ( 13.660361 )
Epoch: 6000  error= 0.25757 ( 13.644603 )
Epoch: 6500  error= 0.25146 ( 13.534495 )
Epoch: 7000  error= 0.25146 ( 13.642559 )
Epoch: 7500  error= 0.24951 ( 13.606575 )
Epoch: 8000  error= 0.25 ( 13.631177 )
Epoch: 8500  error= 0.25024 ( 13.621633 )
Epoch: 9000  error= 0.25 ( 13.641891 )
Epoch: 9500  error= 0.25146 ( 13.718516 )
Epoch: 10000  error= 0.25269 ( 13.659478 )
Epoch: 10500  error= 0.25684 ( 13.576955 )
Epoch: 11000  error= 0.25122 ( 13.564255 )
Epoch: 11500  error= 0.26196 ( 13.5856 )
Epoch

In [92]:
# Regression result